# 📚  Exercise Histopathology Foundation Model

Exercise adapted from Charlotte Bunne, EPFL

This exercise needs to be run on a Modal notebook. 


- [Subtask 1.0: Get familiar with Histopathology images](#Subtask-1.0:-Get-familiar-with-Histopathology-images)
- [Subtask 1.1: Linear Probing based on FM embeddings](#task-11-linear-probing-based-on-fm-embeddings)
- [Subtask 1.2: Tissue-level classification](#task-12-tissue-level-classification)
- [Subtask 1.3: Whole-slide image analysis](#task-13-whole-slide-image-analysis)

## Task 1: Histopathology Foundation Models

In this task we are taking a look at one prominent area of Vision Foundation Models trained using DINOv2: Histopathology.
Since there is a bunch of different clincal "downstream" tasks on which one can possibly utilize large FMs, there is a variety of benchmarks available to, e.g., predict tumor (sub-)types, gene expressions within areas of the tissue, interesting Regions of Interest (ROIs). An overview of a few different and commonly used pathology models can be found here: https://birkhoffkiki.github.io/PathBench/. For this exercise, we are using the Phikon2 model.

In [7]:
!pip install torch transformers datasets Pillow umap-learn
!pip install s3fs tiffile imagecodecs zarr scikit-image
!pip install matplotlib seaborn

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2026.1.0
    Uninstalling fsspec-2026.1.0:
      Successfully uninstalled fsspec-2026.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2026.1.0 requires fsspec==2026.1.0, but you have fsspec 2025.10.0 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.10.0
    Uninstalling fsspec-2025.10.0:
      Successfully uninstalled fsspec-2025.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.5.0 requires fsspec[http]<=2025.10.0,>=2023.1.0, but you have fsspec 2026.1.0 which is incompatible.

[notice

In [8]:
try:
    from umap.umap_ import UMAP
except ImportError:
    from umap import UMAP

In [9]:
from PIL import Image
import torch
from transformers import AutoImageProcessor, AutoModel
import matplotlib.pyplot as plt
from datasets import load_dataset
from tqdm import tqdm
import numpy as np

In [10]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

<a id="task1.0"></a>

### Task 1.0: Getting familiar with Histopathology images.

**Histopathology background:** <br>
Histopathology is the microscopic examination of tissue samples to study the manifestations of disease. It is widely used in medical diagnosis to identify abnormalities such as cancer, inflammation, and infections. <br>
One of the fundamental staining techniques in histopathology is Hematoxylin and Eosin (H&E) staining. This method uses two dyes: hematoxylin and eosin. Hematoxylin stains cell nuclei a purplish-blue color, highlighting DNA and nuclear structures, while eosin stains the cytoplasm and extracellular matrix various shades of pink or red. The color contrast provided by H&E staining allows pathologists to easily differentiate cellular components and assess tissue architecture and pathological changes effectively. <br>
H&E staining is valued for its simplicity, cost-effectiveness, and ability to provide detailed visualization of tissue structure, making it the gold standard in histopathology laboratories globally. <br>

In the following, we are taking a look at H&E stains of lung tissue samples. More precisly, we are trying to label the lung cancer type of the H&E stains to distinguish between Lung adenocarcinoma (LUAD) and Lung squamous cell carcinoma (LUSC). 


In [11]:
# if you get an error here, try setting streaming=False. 
# This will download the dataset locally in advance.
ds_train = load_dataset("dakomura/tcga-ut", "internal", split="train", streaming=True) 

ds_train = ds_train.filter(lambda sample: "Lung" in sample["json"]["label"])

ds_test = load_dataset("dakomura/tcga-ut", "internal", split="test", streaming=True)
ds_test = ds_test.filter(lambda sample: "Lung" in sample["json"]["label"])

Resolving data files:   0%|          | 0/39 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/39 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/39 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/39 [00:00<?, ?it/s]

In [12]:
# Load phikon-v2
processor = AutoImageProcessor.from_pretrained("owkin/phikon-v2", use_fast=True)
model = AutoModel.from_pretrained("owkin/phikon-v2")
model.eval()
model.to(device)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/750 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

Dinov2Model(
  (embeddings): Dinov2Embeddings(
    (patch_embeddings): Dinov2PatchEmbeddings(
      (projection): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): Dinov2Encoder(
    (layer): ModuleList(
      (0-23): 24 x Dinov2Layer(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attention): Dinov2Attention(
          (attention): Dinov2SelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (output): Dinov2SelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (layer_scale1): Dinov2LayerScale()
        (drop_path): Identity()
        (norm2): LayerNorm((1024,),

In [13]:
def load_and_process_image(sample):
    image_bytes = sample["jpg"]
    # image = Image.open(io.BytesIO(image_bytes))
    image = image_bytes
    return {
        "image": image,
        "label": sample["json"]["label"],
        "patient_id": sample["__key__"][:12] # TCGA-XX-XXXX
    }

ds_train_processed = ds_train.map(load_and_process_image, remove_columns=["jpg", "json", "__key__", "__url__"])
ds_test_processed = ds_test.map(load_and_process_image, remove_columns=["jpg", "json", "__key__", "__url__"])


**Exercise 1**: Visualize some of the H&E samples of the dataset and display corresponding the labels and patient ids.

- Are histopathology images different from common images, e.g., as seen in the ImageNet-1k dataset?
- If so, what implications would this have...
    - to data distributions?
    - to general pre-trained FMs for RGB images?
    - to data pre-processing?
    - ...


Visualize some of the crops, print the label as well as as the patient id of the crop.

In [15]:
iter_ds = iter(ds_train_processed)

### YOUR CODE to visualize images here ###
# Hint: iterate over the data loader
# use display from IPython.display to show images in a Jupyter notebook
from IPython.display import display

<a name="task1.1"></a>

### Task 1.1: Linear Probing based on FM embeddings

**Exercise 2**: 
1. Embed the training crops, store the patient ids as well as the labels. 
2. Visualize the embeddings using a UMAP
3. Fit a Logistic Regression (LR) model on the training set
4. Embed the testset crops, and evaluate the LR using accuracy and F1-score.


In [ ]:
ds_train_processed = ds_train_processed.with_format("torch")
ds_test_processed = ds_test_processed.with_format("torch")

BATCH_SIZE = 32
EMBEDDING_SIZE = 1024
NR_BATCHES = 100

dl_train = torch.utils.data.DataLoader(
    ds_train_processed,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=8,
)

In [ ]:
embeddings = torch.zeros((BATCH_SIZE * NR_BATCHES, EMBEDDING_SIZE))
labels = []
patient_ids = []

with torch.inference_mode():
    with torch.autocast(device.type, torch.bfloat16):
        for i, batch in enumerate(tqdm(dl_train, total=NR_BATCHES)):
            inputs = processor(batch["image"], return_tensors="pt").to(device)

            ### YOUR CODE ###
            # Hint: use model to get embeddings
            # get the last hidden state and pool it to get a single embedding per image
            ### END YOUR CODE ###


            if i == (NR_BATCHES - 1):
              break

**Exercise 3**:

Now that we have encoded our lung histopathology crops, visualize them in a UMAP.
Using the same umap embedding, color the plot once by patient ids and once by tumor type.

Create a second UMAP in which you visualize the embedded crops from only four patients, e.g. ['TCGA-33-4582', 'TCGA-55-8614', 'TCGA-77-8138', 'TCGA-71-8520'].

- What do you see in the UMAPs?
    - Are there overlapping classes?
    - If so why, if not why not?
    - How nicely are the cancer types clustering?
    - Do we also see clustering in the patients? Why could this be?

In [ ]:
# Apply UMAP
import seaborn as sns
labels = np.array(labels)
patient_ids = np.array(patient_ids)

### YOUR CODE ###

# Use below snippet from the UMAP library
# UMAP(n_neighbors=30, min_dist=0.3, n_components=2, random_state=42)


Now, we can do some logistic regression based on the FM embeddings to differentiate between Lung Adenocarcinoma (LUAD) and Lung Squamous Cell Carcinoma ( LUSC).

First, train a Logistic Regression classifier, afterwards embed the first 100 batches with a batch size of 32 of the test dataset defined above and calculate the accuracy as well as the f1-score on it. Additionally, save the patient ids for the next task.

In [ ]:
from sklearn.linear_model import LogisticRegression

### YOUR CODE ###

# Use a logistic regression model from sklearn to classify the embeddings
# Train on the embeddings from the training set above and the labels

In [ ]:
dl_test = torch.utils.data.DataLoader(
    ds_test_processed,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=8,
)

In [ ]:
### YOUR CODE ###

## now do the same what you did to calculate embeddings for train set, but for test set

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

### YOUR CODE ###

# Use the trained logistic regression model to predict on the test embeddings
# Calculate accuracy, F1-score and confusion matrix

<a id="task1.2"></a>


### Task 1.2: Tissue-level classification

Until now, we did everything on a crop level (256x256 pixels) extracted from a larger pathology whole slide that often spans more than 10,000 x 10,000 pixels.
As we are interested in a tissue-level and not crop level task, it would make more sense to derive predictions on a patient level.

We implement majority voting as a simple baseline to derive tissue-level prediction from the logisitic regression based test-set predictions





In [ ]:
import pandas as pd

df = pd.DataFrame({"pid": patient_ids_test, "prediction": lr_predictions, "label": labels_test})
df_majority_voting = df.groupby("pid").agg({"prediction":pd.Series.mode, "label":"first"})

predictions_tissue = df_majority_voting["prediction"].values
labels_tissue = df_majority_voting["label"].values

# resolve ties
predictions_tissue = [p[0] if isinstance(p, np.ndarray) else p for p in predictions_tissue]

Now calculate tissue-level accuracies and f1-score.
- Would you expect improving performances? Why?

In [ ]:
### YOUR CODE ###

<a id="task1.3"></a>


### Task 1.3: Whole-slide image analysis

So far we have taken a look at tissue-level downstream tasks where we were interested in global labels. We can also take a look at more local labels in a whole slide image (WSI).

In this task, we are using a WSI originating from a colorectal cancer (CRC) patient. A visualization of the WSI can be found [here](https://www.cycif.org/data/orion-crc/P37_S38-CRC10#s=0#w=0#g=5#m=-1#a=-100_-100#v=0.4618_0.8898_0.6185#o=-100_-100_1_1#p=Q).



In [ ]:
import s3fs
import os

fs = s3fs.S3FileSystem(anon=True)
s3_path = 'lin-2023-orion-crc/data/CRC10/18459_LSP10452_US_SCAN_OR_001__091355-registered.ome.tif'

# Local filename to save the image
local_filename = 'crc10_he_wsi.ome.tif'

if not os.path.exists(local_filename):
    # Download the file from S3 to local filesystem
    fs.get(s3_path, local_filename)

print(f'Downloaded {local_filename}')

Loading the image. To save some RAM and compute time, we only take a look at a 10,080 x 10,080 subpart of the WSI.

In [ ]:
import tifffile

region = tifffile.imread(local_filename)
region = region[15000:25080, 5000:15080]
plt.imshow(region)

Now, embed the tissue by iterating over the image using a crop size of 224x224 without overlap, store the cls tokens and the local patch tokens of each of the crops.

Afterwards, take the top left and the bottom right cls embedding as well as the mean of local patch embeddings of the top left as well as bottom right crop. These will serve as "prototypes". Cacluate the cosine similarities between all cls tokens and the cls-based prototypes and the patches and the patch-based prototypes, respectively.

Hint: the patch size of the Phikon2 model is 16x16.

**Exercise 4**:
1. Implement the sliding window embedding 
2. Calculate cosine similarities of a prototype of the image and all other embeddings
3. Visualize the cosine similarities

In [ ]:
embeddings_cls = torch.zeros((45,45, EMBEDDING_SIZE))
embeddings_ps = torch.zeros((630,630, EMBEDDING_SIZE))

region_x, region_y = region.shape[:2]

with torch.inference_mode():
    with torch.autocast(device.type, torch.bfloat16):
        ### YOUR CODE ###
        # inputs = processor(crop, return_tensors="pt").to(device)
        # outputs = model(**inputs)
        # cls_tokens = outputs.last_hidden_state[:, 0, :].cpu()
        # embeddings_cls[x,y] = cls_tokens
        # patch_tokens = outputs.last_hidden_state[:, 1:, :].cpu()
        # patch_tokens = patch_tokens.reshape((14,14, EMBEDDING_SIZE))
        # embeddings_ps[x*14 : (x+1)*14, y*14 : (y+1) * 14] = patch_tokens
        pass


In [ ]:
import torch
import torch.nn.functional as F

# Example Top left token
# For the cls tokens first
cls_00 = embeddings_cls[0, 0]  # shape: (EMBEDDING_SIZE,)
cls_00 = cls_00.unsqueeze(0)  # shape: (1, EMBEDDING_SIZE)

vectors_all_cls = embeddings_cls.view(-1, embeddings_cls.shape[2])
cosine_sim_cls_00 = F.cosine_similarity(cls_00, vectors_all_cls, dim=1)
cosine_sim_cls_00 = cosine_sim_cls_00.reshape(embeddings_cls.shape[:2])

# For the ps tokens 
ps_00 = embeddings_ps[0, 0]  # shape: (EMBEDDING_SIZE,)
ps_00 = torch.mean(embeddings_ps[:14, :14], dim=(0,1))
ps_00 = ps_00.unsqueeze(0)  # shape: (1, EMBEDDING_SIZE)

vectors_all_ps = embeddings_ps.view(-1, embeddings_ps.shape[2])
cosine_sim_ps_00 = F.cosine_similarity(ps_00, vectors_all_ps, dim=1)
cosine_sim_ps_00 = cosine_sim_ps_00.reshape(embeddings_ps.shape[:2])

Visualize the cosine similarities on top of the image between the selected prototypes and all other embeddings (once for the cls tokens, once for the patch tokens).

- What do you observe?
- What could this be useful for in a clinical setting?

In [ ]:
from skimage.transform import resize



cosine_sim_cls_00_resized = resize(np.array(cosine_sim_cls_00), (region_x, region_y))
cosine_sim_cls_nn_resized = resize(np.array(cosine_sim_cls_nn), (region_x, region_y))

cosine_sim_ps_00_resized = resize(np.array(cosine_sim_ps_00), (region_x, region_y))
cosine_sim_ps_nn_resized = resize(np.array(cosine_sim_ps_nn), (region_x, region_y))


plt.imshow(region[::10, ::10])
plt.axis('off')
plt.show()

fig, axs = plt.subplots(2,3, figsize=(12, 5))
axs = axs.flatten()
axs[0].imshow(region[:224, :224])
axs[0].set_title("Top left crop (224x224)")
axs[1].imshow(region[::10, ::10])
axs[1].imshow(cosine_sim_cls_00_resized[::10, ::10], alpha=0.3, cmap="jet")
axs[2].imshow(region[::10, ::10])
axs[2].imshow(cosine_sim_ps_00_resized[::10, ::10], alpha=0.3, cmap="jet")
axs[3].imshow(region[-224:, -224:])
axs[3].set_title("Bottom right crop (224x224)")
axs[4].imshow(region[::10, ::10])
axs[4].imshow(cosine_sim_cls_nn_resized[::10, ::10], alpha=0.3, cmap="jet")
axs[5].imshow(region[::10, ::10])
axs[5].imshow(cosine_sim_ps_nn_resized[::10, ::10], alpha=0.3, cmap="jet")

axs[1].set_title("Cos. sim (CLS)")
axs[2].set_title("Cos. sim (Patch)")

for ax in axs:
    ax.axis('off')